In [1]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import s3fs

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials f

In [2]:
importlib.reload(afqd)

<module 'AFQ.data' from '/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/AFQ/data.py'>

In [3]:
ixi = afqd.S3BIDSStudy(
    study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=5,
    anon=False, use_participants_tsv=False, random_seed=42
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  1.3s


In [4]:
ixi.non_sub_s3_keys

{'raw': ['ixi.bids/dataset_description.json',
  'ixi.bids/dwi.bval',
  'ixi.bids/dwi.bvec'],
 'derivatives': []}

In [5]:
ixi.download('./ixi-bids')

In [6]:
import os.path as op

fs = s3fs.S3FileSystem()

for fn in ixi.non_sub_s3_keys['raw']:
    fs.get(fn, op.join('.', 'ixi-bids', op.basename(fn)))

In [7]:
s0 = ixi.subjects[0]
s1 = ixi.subjects[1]

In [8]:
import os.path as op

op.commonpath(list(s0.files['raw'].values()) + list(s1.files['raw'].values()))

'/Users/richford/projects/neuro/hbn-hub/notebooks/ixi-bids'

In [9]:
def preprocess_ixi_subject(subject_id):
    import AFQ.data as afqd
    import os
    import subprocess
    from s3fs import S3FileSystem
    
    ixi = afqd.afqd.S3BIDSStudy(
        study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=subject_id,
        anon=False, use_participants_tsv=False
    )
        
    ixi.download("./ixi")
    
    fs = s3fs.S3FileSystem()
        
    for fn in ixi.non_sub_s3_keys['raw']:
        if 'bval' in fn or 'bvec' in fn:
            fs.get(fn, os.path.join('.', 'ixi-bids', op.basename(fn)))
    
    subprocess.run(
        ["qsiprep",
         "./ixi",
         "./ixi-preproc",
         subject_id],
        shell=True,
        check=True,
    )

    def upload_dir(path):
        output_dir = 'ixi.bids/derivatives/qsiprep_ck'
        if fs.exists(output_dir):
            walk_dir = os.walk(path)
            _, dirs, files = next(walk_dir)
            for fn in dirs + files:
                fs.put(os.path.join(path, fn), output_dir, recursive=True)
        else:
            fs.put(path, output_dir, recursive=True)
            
    upload_dir("./ixi-preproc")

# Skip this section because the Docker image is already on AWS ECR
The cells to skip are commented out

In [10]:
# di = ck.DockerImage(
#     func=preprocess_ixi_subject,
#     base_image="pennbbl/qsiprep",
#     github_installs="https://github.com/richford/pyAFQ.git@s3-bids-fetch",
# )

In [11]:
# di.build_path

#### Instruction

Go to that directory and edit the Dockerfile to include `--ignore-installed` to the `pip install` commands.

In [12]:
# di.build(tags=["ixi_preproc_20200708"])

In [13]:
# import logging
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

In [14]:
# di.images

In [15]:
# di.repo_uri

In [16]:
# ck.get_ecr_repo()

In [17]:
# repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [18]:
# repo.repo_uri

In [19]:
# # This command takes a few hours because the docker image is large
# di.push(repo=repo)

# Pick up here, retrieving the Docker image from AWS ECR

In [20]:
di = ck.DockerImage(name='preprocess-ixi-subject')

# Try the Amazon ECS-Optimized Amazon Linux 2 AMI

It might allow a larger docker image size

In [76]:
ck.get_region()

'us-east-2'

In [110]:
ec2 = ck.aws.clients['ec2']
response = ec2.describe_images(Owners=["amazon"])

In [111]:
# ecs_image = [
#     image for image in response["Images"]
#     if image["ImageId"] == "ami-0466acdbae3d9cc42"
# ][0]
# ecs_image

In [112]:
ecs_images = sorted(
    [
        image for image in response["Images"]
        if image.get("Description") is not None
        and "amazon linux ami 2" in image["Description"].lower()
        and "x86_64 ecs hvm gp2" in image["Description"].lower()
        and "gpu" not in image["Name"].lower()
        and len(image["BlockDeviceMappings"]) == 1
    ],
    key=lambda image: image["CreationDate"],
    reverse=True
)
image = ecs_images[0]
ami_id = image["ImageId"]
print(image)
print(ami_id)

{'Architecture': 'x86_64', 'CreationDate': '2020-07-09T16:30:21.000Z', 'ImageId': 'ami-0fca7970ac53c18de', 'ImageLocation': 'amazon/amzn2-ami-ecs-hvm-2.0.20200708-x86_64-ebs', 'ImageType': 'machine', 'Public': True, 'OwnerId': '591542846629', 'State': 'available', 'BlockDeviceMappings': [{'DeviceName': '/dev/xvda', 'Ebs': {'DeleteOnTermination': True, 'SnapshotId': 'snap-0b09d0b9dc29b6a1f', 'VolumeSize': 30, 'VolumeType': 'gp2', 'Encrypted': False}}], 'Description': 'Amazon Linux AMI 2.0.20200708 x86_64 ECS HVM GP2', 'EnaSupport': True, 'Hypervisor': 'xen', 'ImageOwnerAlias': 'amazon', 'Name': 'amzn2-ami-ecs-hvm-2.0.20200708-x86_64-ebs', 'RootDeviceName': '/dev/xvda', 'RootDeviceType': 'ebs', 'SriovNetSupport': 'simple', 'VirtualizationType': 'hvm'}
ami-0fca7970ac53c18de


In [113]:
ec2 = ck.aws.clients['ec2']
response = ec2.describe_images(
    Filters=[{
        'Name': 'description',
        'Values': ['Amazon ECS-Optimized Amazon Linux 2 AMI']
    }]
)
print(response["Images"][0]["ImageId"])

ami-0888e1d90572c95b6


In [114]:
knot = ck.Knot(
    name="ixi_preproc_20200713_arh_v0",
    docker_image=di,
    pars_policies=('AmazonS3FullAccess',),
    resource_type="SPOT",
    bid_percentage=100,
    memory=64000,
    job_def_vcpus=32,
    image_id=ami_id
)

In [115]:
sub_ids = [s.subject_id for s in ixi.subjects]
sub_ids

['sub-IXI538', 'sub-IXI631', 'sub-IXI039', 'sub-IXI108', 'sub-IXI139']

In [116]:
result_futures = knot.map(sub_ids)

In [118]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
ae6f6a8b-bf8d-46fe-a816-1f2ba928d93c        ixi-preproc-20200713-arh-v0-0        FAILED   


In [104]:
knot.clobber(clobber_pars=True)

In [ ]:
print([j.status for j in knot.jobs])

In [ ]:
knot.clobber(clobber_pars=True)